### Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
import re
# import contractions
import string
from bs4 import BeautifulSoup
# import requests
# from collections import Counter
# from num2words import num2words
from wordcloud import WordCloud, ImageColorGenerator
from nltk.tokenize import word_tokenize
from tqdm import tqdm
tqdm.pandas()
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
# import spacy
import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

import nltk
import warnings
warnings.filterwarnings("ignore")

from nltk.corpus import stopwords
from google.colab import drive
drive.mount('/content/drive')

pd.set_option('max_colwidth', 99999)

[nltk_data] Downloading package wordnet to /root/nltk_data...


Mounted at /content/drive


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.7 MB/s eta 0:00:00


### Dataset Retrieval

In [ ]:
from datasets import load_dataset
dataset = load_dataset("lince",'lid_hineng', download_mode="force_redownload")
print(f"Train dataset size: {len(dataset['train'])}")
print(f"Test dataset size: {len(dataset['test'])}")

Generating train split:   0%|          | 0/4823 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/744 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1854 [00:00<?, ? examples/s]

Train dataset size: 4823
Test dataset size: 1854


In [ ]:
dataset['train']

Dataset({
    features: ['idx', 'words', 'lid'],
    num_rows: 4823
})

### Prepare data

In [ ]:
trid = pd.Series(dataset['train']['idx'])
trwo = pd.Series(dataset['train']['words'])
trli = pd.Series(dataset['train']['lid'])
train = pd.DataFrame({'id':trid, 'words':trwo, 'lid':trli})

vaid = pd.Series(dataset['validation']['idx'])
vawo = pd.Series(dataset['validation']['words'])
vali = pd.Series(dataset['validation']['lid'])
valid = pd.DataFrame({'id':vaid, 'words':vawo, 'lid':vali})

teid = pd.Series(dataset['test']['idx'])
tewo = pd.Series(dataset['test']['words'])
test = pd.DataFrame({'id':teid, 'words':tewo})

In [ ]:
train

,id,words,lid
0,0,"[Good, vibe, tribe, .]","[lang1, lang1, lang1, other]"
1,1,"[beware, #pmjohnkey, .., ., maintain, distance...","[lang1, ne, other, other, lang1, lang1, lang1,..."
2,2,"[Angry, young, men, sir, ji]","[lang1, lang1, lang1, lang1, lang2]"
3,3,"[Girlfriend, 's, Guide, To, Divorce, is, the, ...","[lang1, lang1, lang1, lang1, lang1, lang1, lan..."
4,4,"[Ajay, thakur, ,, woww, this, is, why, kabaddi...","[ne, ne, other, lang1, lang1, lang1, lang1, la..."
...,...,...,...
4818,4818,"[Hats, Off, !!]","[lang1, lang1, other]"
4819,4819,"[par, toda, dhyan, se, sahab, 62, me, bhi, en,...","[lang1, lang2, lang2, lang2, lang2, other, lan..."
4820,4820,"[Kya, saalo, kb, se, daily, dekhta, ho, ki, sa...","[lang2, lang2, lang2, lang2, lang1, lang2, lan..."
4821,4821,"[hi, amir, my, name, nadim, khan]","[lang2, ne, lang1, lang1, ne, ne]"


In [ ]:
copytrain = train.copy()

In [ ]:
copytrain = copytrain[['id', 'words', 'lid']]

In [ ]:
copytrain['old_words'] = copytrain['words']
copytrain['old_lid'] = copytrain['lid']

In [ ]:
def remove_video_tags(tokens):
    '''
    remove_video_tags:
    Removes video tags from a list of words.
    Args:
      tokens (list) : List of words with video tags
    Returns:
      filtered_tokens (list) : List of words without video tags
    '''
    clean = re.compile('[[.*?]]')
    filtered_tokens = [re.sub(clean, '', token) for token in tokens]
    return filtered_tokens

def remove_multiple_letters(tokens):
    '''
    remove_multiple_letters:
    Replace words with more than 2 consecutive letters with 2 consecutive letters in a list of words.
    Args:
      tokens (list) : List of words to process
    Returns:
      processed_tokens (list) : List of words with replacements
    '''
    processed_tokens = [re.sub(r'(.)\1{2,}', r'\1\1', token) for token in tokens]
    return processed_tokens

def joinstr(tokens):
    '''
    joinstr:
    Joins a list of words with periods.
    Args:
      tokens (list) : List of words
    Returns:
      joined_str (str) : String as list joined with periods
    '''
    joined_str = '.'.join(tokens)
    return joined_str

def remove_html_tags(tokens):
    '''
    remove_html_tags:
    Removes HTML tags from a list of words.
    Args:
      tokens (list) : List of words with HTML tags
    Returns:
      filtered_tokens (list) : List of words without HTML tags
    '''
    clean = re.compile('<.*?>')
    filtered_tokens = [re.sub(clean, '', token) for token in tokens]
    return filtered_tokens

def remove_url(tokens):
    '''
    remove_url:
    Removes words containing 'http' or 'www' from a list of words.
    Args:
      tokens (list) : List of words
    Returns:
      filtered_tokens (list) : List of words with 'http' or 'www' removed
    '''
    filtered_tokens = [token if 'http' not in token and 'www' not in token else '' for token in tokens]
    return filtered_tokens

def remove_attherate(tokens):
    '''
    remove_url:
    Removes words containing 'http' or 'www' from a list of words.
    Args:
      tokens (list) : List of words
    Returns:
      filtered_tokens (list) : List of words with 'http' or 'www' removed
    '''
    filtered_tokens = [token if '@' not in token else '' for token in tokens]
    return filtered_tokens

In [ ]:
def find_removed_indices(original_list, modified_list):
    # Find indices of blank strings in the modified list
    removed_indices = [i for i, item in enumerate(modified_list) if item == ""]
    return removed_indices

def del_list_indexes(l, id_to_del):
    somelist = [i for j, i in enumerate(l) if j not in id_to_del]
    return somelist

def remove_blank_strings(tokens):
    tokens = [x for x in tokens if x]
    return tokens

def lower_case(tokens):
    tokens = [x.lower() for x in tokens]
    return tokens

def blank_consecutive_duplicates(input_list, blank_value=''):
    if not input_list:
        return input_list  # Return an empty list if the input list is empty
    # Initialize a new list with the first element from the input list
    result_list = [input_list[0]]
    # Iterate through the input list, starting from the second element
    for i in range(1, len(input_list)):
        if input_list[i] == input_list[i - 1]:
            result_list.append(blank_value)
        else:
            result_list.append(input_list[i])
    return result_list

def replace_words_and_tags(row):
    words = row['words']
    lid = row['lid']

    for i in range(len(lid)):
        if lid[i] in drop_lids:
            words[i] = ''  # Replace the word with a blank string
            lid[i] = ''   # Replace the tag with a blank string

    return pd.Series({'words': words, 'lid': lid})

def update_words_lid(copytrain):
  for i in range(len(copytrain['words'])):
    m = find_removed_indices(copytrain['words'][i], copytrain['n_words'][i])
    copytrain['lid'][i] = del_list_indexes(copytrain['lid'][i], m)
  copytrain['n_words'] = copytrain['n_words'].apply(lambda x: remove_blank_strings(x))
  copytrain['words'] = copytrain['n_words']

In [ ]:
copytrain['n_words'] = copytrain['words'].apply(lambda x: remove_video_tags(x))
update_words_lid(copytrain)

In [ ]:
copytrain['n_words'] = copytrain['words'].apply(lambda x: remove_html_tags(x))
update_words_lid(copytrain)

In [ ]:
copytrain['n_words'] = copytrain['words'].apply(lambda x: remove_url(x))
update_words_lid(copytrain)

In [ ]:
copytrain['n_words'] = copytrain['words'].apply(lambda x: remove_attherate(x))
update_words_lid(copytrain)

In [ ]:
copytrain['n_words'] = copytrain['words'].apply(lambda tokens: [re.sub(r'[^A-Za-z ]+', '', word) for word in tokens])
update_words_lid(copytrain)

In [ ]:
copytrain['n_words'] = copytrain['words'].apply(lambda x: remove_multiple_letters(x))
update_words_lid(copytrain)

In [ ]:
copytrain['n_words'] = copytrain['words'].apply(lambda x: lower_case(x))
update_words_lid(copytrain)

In [ ]:
copytrain['n_words'] = copytrain['words'].apply(lambda x: blank_consecutive_duplicates(x))
update_words_lid(copytrain)

In [ ]:
lid = list(copytrain['lid'])
wordlist = list(copytrain['words'])
flat_lid = [item for sublist in lid for item in sublist]
flat_wordlist = [item for sublist in wordlist for item in sublist]
key = list(set(flat_lid))
pd.Series(flat_wordlist)
pd.Series(flat_lid)
flat = pd.DataFrame({'words':flat_wordlist, 'lid':flat_lid})
for i in key:
  print(str(i) + ': ' +str(flat['words'][flat['lid']==i].value_counts().sum()))
  print(flat['words'][flat['lid']==i].value_counts()[:20])
  print('-------')


other: 392
p         62
d         36
oh        24
th        18
ha        16
nd        14
s         13
h         11
haha      10
am        10
hahaha     9
m          8
pm         7
a          5
uff        5
yup        4
xd         4
v          4
hmm        4
cr         4
Name: words, dtype: int64
-------
ne: 6288
india       475
virat       254
modi        241
salman      140
dhoni       125
kohli       120
khan        118
pakistan     98
aamir        90
russia       72
diwali       54
priyanka     54
indian       49
ms           42
modiji       42
narendra     34
dangal       33
delhi        31
sultan       31
amir         30
Name: words, dtype: int64
-------
unk: 10
y         1
gavi      1
kamboj    1
prash     1
gant      1
saa       1
lb        1
m         1
been      1
cajun     1
Name: words, dtype: int64
-------
fw: 405
halamadrid    6
na            5
thalaivar     5
dar           5
ki            4
hoy           4
tere          4
thala         3
to            3
mama          3
ka

In [ ]:
drop_lids = ['ambiguous','unk', 'mixed', 'other']

In [ ]:
copytrain[['n_words','lid']] = copytrain.apply(replace_words_and_tags, axis=1)

In [ ]:
update_words_lid(copytrain)

In [ ]:
copytrain['words'] = copytrain['n_words']

Merging fw with lang2 and ne with lang1

In [ ]:
for i in range(len(copytrain['lid'])):
  for j in range(len(copytrain['lid'][i])):
    if copytrain['lid'][i][j] == 'fw':
      copytrain['lid'][i][j] = 'lang2'
    if copytrain['lid'][i][j] == 'ne':
      copytrain['lid'][i][j] = 'lang1'

In [ ]:
lid = list(copytrain['lid'])
wordlist = list(copytrain['words'])
flat_lid = [item for sublist in lid for item in sublist]
flat_wordlist = [item for sublist in wordlist for item in sublist]
key = list(set(flat_lid))
pd.Series(flat_wordlist)
pd.Series(flat_lid)
flat = pd.DataFrame({'words':flat_wordlist, 'lid':flat_lid})
for i in key:
  print(str(i) + ': ' +str(flat['words'][flat['lid']==i].value_counts().sum()))
  print(flat['words'][flat['lid']==i].value_counts()[:20])
  print('-------')

lang2: 19481
hai     564
ki      383
ji      310
ko      302
bhai    290
ho      270
ke      268
ka      255
to      247
h       247
me      230
se      221
bhi     218
aap     204
hi      196
nahi    167
aur     164
k       152
koi     130
kya     125
Name: words, dtype: int64
-------
lang1: 61073
the      1468
to       1263
you      1163
i        1135
and      1048
of        970
in        937
is        881
a         846
sir       668
u         659
for       651
are       484
india     476
it        431
we        413
this      404
love      368
best      359
all       357
Name: words, dtype: int64
-------


In [ ]:
copytrain['final_lid_str'] = ''
copytrain['final_lid'] = ''

In [ ]:
def assign_language(row):
    if 'lang1' in row['lid'] and 'lang2' in row['lid']:
        return 'bi'
    elif 'lang1' in row['lid']:
        return 'eng'
    elif 'lang2' in row['lid']:
        return 'hin'
    else:
        return None  # Or any other default value you prefer

In [ ]:
copytrain['final_lid'] = copytrain.apply(assign_language, axis=1)

In [ ]:
copytrain[['words','lid','final_lid']]

,words,lid,final_lid
0,"[good, vibe, tribe]","[lang1, lang1, lang1]",eng
1,"[beware, pmjohnkey, maintain, distance, from, ...","[lang1, lang1, lang1, lang1, lang1, lang1, lan...",eng
2,"[angry, young, men, sir, ji]","[lang1, lang1, lang1, lang1, lang2]",bi
3,"[girlfriend, s, guide, to, divorce, is, the, b...","[lang1, lang1, lang1, lang1, lang1, lang1, lan...",eng
4,"[ajay, thakur, woww, this, is, why, kabaddi, i...","[lang1, lang1, lang1, lang1, lang1, lang1, lan...",bi
...,...,...,...
4818,"[hats, off]","[lang1, lang1]",eng
4819,"[par, toda, dhyan, se, sahab, me, bhi, en, log...","[lang1, lang2, lang2, lang2, lang2, lang2, lan...",bi
4820,"[kya, saalo, kb, se, daily, dekhta, ho, ki, sa...","[lang2, lang2, lang2, lang2, lang1, lang2, lan...",bi
4821,"[hi, amir, my, name, nadim, khan]","[lang2, lang1, lang1, lang1, lang1, lang1]",bi


In [ ]:
def prepdata(train):
  copytrain = train.copy()
  copytrain = copytrain[['id', 'words', 'lid']]
  copytrain['old_words'] = copytrain['words']
  copytrain['old_lid'] = copytrain['lid']
  copytrain['n_words'] = copytrain['words'].apply(lambda x: remove_video_tags(x))
  update_words_lid(copytrain)
  copytrain['n_words'] = copytrain['words'].apply(lambda x: remove_html_tags(x))
  update_words_lid(copytrain)
  copytrain['n_words'] = copytrain['words'].apply(lambda x: remove_url(x))
  update_words_lid(copytrain)
  copytrain['n_words'] = copytrain['words'].apply(lambda x: remove_attherate(x))
  update_words_lid(copytrain)
  copytrain['n_words'] = copytrain['words'].apply(lambda tokens: [re.sub(r'[^A-Za-z ]+', '', word) for word in tokens])
  update_words_lid(copytrain)
  copytrain['n_words'] = copytrain['words'].apply(lambda x: remove_multiple_letters(x))
  update_words_lid(copytrain)
  copytrain['n_words'] = copytrain['words'].apply(lambda x: lower_case(x))
  update_words_lid(copytrain)
  copytrain['n_words'] = copytrain['words'].apply(lambda x: blank_consecutive_duplicates(x))
  update_words_lid(copytrain)
  drop_lids = ['ambiguous','unk', 'mixed', 'other']
  copytrain[['n_words','lid']] = copytrain.apply(replace_words_and_tags, axis=1)
  update_words_lid(copytrain)
  copytrain['words'] = copytrain['n_words']
  for i in range(len(copytrain['lid'])):
    for j in range(len(copytrain['lid'][i])):
      if copytrain['lid'][i][j] == 'fw':
        copytrain['lid'][i][j] = 'lang2'
      if copytrain['lid'][i][j] == 'ne':
        copytrain['lid'][i][j] = 'lang1'
  copytrain['final_lid'] = ''
  copytrain['final_lid'] = copytrain.apply(assign_language, axis=1)
  return copytrain[['words','lid','final_lid']]

In [ ]:
validation_data = prepdata(valid)

In [ ]:
copytrain['final_lid'].value_counts()

eng    2478
bi     2236
hin     103
Name: final_lid, dtype: int64

In [ ]:
dx = copytrain[['words','lid','final_lid']]

In [ ]:
dx

,words,lid,final_lid
0,"[good, vibe, tribe]","[lang1, lang1, lang1]",eng
1,"[beware, pmjohnkey, maintain, distance, from, ...","[lang1, lang1, lang1, lang1, lang1, lang1, lan...",eng
2,"[angry, young, men, sir, ji]","[lang1, lang1, lang1, lang1, lang2]",bi
3,"[girlfriend, s, guide, to, divorce, is, the, b...","[lang1, lang1, lang1, lang1, lang1, lang1, lan...",eng
4,"[ajay, thakur, woww, this, is, why, kabaddi, i...","[lang1, lang1, lang1, lang1, lang1, lang1, lan...",bi
...,...,...,...
4818,"[hats, off]","[lang1, lang1]",eng
4819,"[par, toda, dhyan, se, sahab, me, bhi, en, log...","[lang1, lang2, lang2, lang2, lang2, lang2, lan...",bi
4820,"[kya, saalo, kb, se, daily, dekhta, ho, ki, sa...","[lang2, lang2, lang2, lang2, lang1, lang2, lan...",bi
4821,"[hi, amir, my, name, nadim, khan]","[lang2, lang1, lang1, lang1, lang1, lang1]",bi


In [ ]:
test_data = prepdata(valid)
test_data

,words,lid,final_lid
0,"[loved, the, ending, i, could, have, offered, ...","[lang1, lang1, lang1, lang1, lang1, lang1, lan...",eng
1,"[viru, bahi, tusi, legend, ho, respect]","[lang1, lang2, lang2, lang1, lang2, lang1]",bi
2,"[we, the, fans, luv, you]","[lang1, lang1, lang1, lang1, lang1]",eng
3,"[jahaa, panaa, tussii, gr, ho, gr, inspiration...","[lang2, lang2, lang2, lang1, lang2, lang1, lan...",bi
4,"[same, as, u, say, we, should, never, ignore, ...","[lang1, lang1, lang1, lang1, lang1, lang1, lan...",eng
...,...,...,...
739,"[academicofficefail, i, appeared, for, es, end...","[lang1, lang1, lang1, lang1, lang1, lang1, lan...",eng
740,"[just, move, ahead, we, r, proud, of, u, kamiy...","[lang1, lang1, lang1, lang1, lang1, lang1, lan...",bi
741,"[i, just, dunno, how, to, talk, to, girls, but...","[lang1, lang1, lang1, lang1, lang1, lang1, lan...",bi
742,"[hope, to, have, a, book, release, in, chandig...","[lang1, lang1, lang1, lang1, lang1, lang1, lan...",eng


### Post, Dataset, PyTorchDataset, and BatchSampler classes

In [ ]:
#!/usr/bin/env python

from collections import Counter
from typing import Callable

import torch
from torch.utils.data import Sampler, Dataset
from torchtext.data.functional import load_sp_model, sentencepiece_numericalizer
from random import shuffle


class Post:
    """A single social media data instance consisting of
    a list of word tokens and a list of language tags"""

    def __init__(self, words: list[str], langs: list[str]):
        self.words = words
        self.langs = langs

    def __getitem__(self, idx):
        return self.words[idx], self.langs[idx]

    def __len__(self):
        return len(self.words)


class LIDDataset(Dataset):
    def __init__(self, dataset):
        self.data: list[Post] = dataset
        self.sp_model = load_sp_model('/content/spm_user.model')
        self.subword_to_idx: Callable = sentencepiece_numericalizer(self.sp_model)
        # self.lang_to_idx: dict[str, int] = {'bn': 0, 'en': 1, 'univ': 2,
                                            # 'ne': 3, 'hi': 4, 'acro': 5, 'mixed': 6, 'undef': 7}
        self.lang_to_idx: dict[str, int] = {'lang1':0 , 'lang2': 1}
        self.weight_dict = self.make_weight_dict()

    def make_weight_dict(self) -> dict:
        """
        Instantiates the weight dict for this dataset
        The formula used is weight = most_frequent/lang_freq.
        Such that the most frequent has a frequency of 1
        :return: A dict with a mapping from a language to weight
        """
        weight_dict = None
        if len(self.data) > 0:
            frequency_dict = {}
            label_counts = Counter([lang for sentence in self.data for lang in sentence.langs])
            for label in self.lang_to_idx.keys():
                frequency_dict[label] = label_counts[label]
            most_frequent = max(frequency_dict.values())
            weight_dict = {label: (most_frequent / frequency_dict[label]) if frequency_dict[label] != 0 else 0
                           for label in frequency_dict.keys()}
        return weight_dict

    def __getitem__(self, idx) -> tuple[list[str], list[str]]:
        idx_item = self.data[idx]
        assert len(idx_item.words) == len(idx_item.langs)
        return idx_item.words, idx_item.langs

    def __len__(self):
        return len(self.data)

    def get_tag_set(self) -> list:
        """returns ordered list of language labels in the dataset
        Returns:
            list -- Ordered list of language labels
        """
        langs = list(self.lang_to_idx.keys())
        langs.sort()
        return langs

    def get_lang_to_idx(self) -> dict:
        """get dict from lang to id, ordered alphabetically
        Returns:
            dict -- For converting language code to an id
        """
        lang_to_idx = {}
        for lang in self.get_tag_set():
            lang_to_idx[lang] = len(lang_to_idx)
        return lang_to_idx


class PyTorchLIDDataSet(Dataset):
    """
    PyTorch-specific wrapper that converts items to PyTorch tensors.
    """

    def __init__(self, decoree: LIDDataset):
        self.data = []
        self.all_post_lens = []
        if decoree is not None:
            self.decoree = decoree
        self.subword_to_idx = decoree.subword_to_idx
        self.lang_to_idx = decoree.lang_to_idx
        self.tensorify_all()

    def __getitem__(self, idx):
        if not isinstance(idx, list):
            return self.data[idx]
        txt = []
        mask = []
        label = []
        for i in idx:
            item = self.data[i]
            txt.append(item[0])
            mask.append(item[1])
            label.append(item[2])

        txts = torch.stack(txt)
        masks = torch.stack(mask)
        labels = torch.stack(label)

        return txts, masks, labels

    def make_weight_dict(self) -> dict:
        return self.decoree.make_weight_dict()

    def __len__(self):
        return len(self.data)

    def get_tag_set(self) -> list:
        return self.decoree.get_tag_set()

    def get_lang_to_idx(self) -> dict:
        """get dict from lang to id, ordered alphabetically
        Returns:
            dict -- For converting language code to an id
        """
        return self.decoree.get_lang_to_idx()

    def tensorify(self, data_point: tuple[list[str], list[str]]):
        words, langs = data_point
        self.all_post_lens.append(len(words))
        word_id = list(self.subword_to_idx(words))
        lang_id = [self.lang_to_idx[lang] for lang in langs]

        # The first subword is assigned the true label, all other subwords are assigned the dummy label -1
        lang_id_pad = [[lang_id[word_num]] + [-1] * (len(word_id[word_num]) - 1) for word_num in range(len(word_id))]

        word_ids_flat = [w_id for word in word_id for w_id in word]
        lang_ids_flat = [l_id for lang in lang_id_pad for l_id in lang]

        mask_nest = [[True] + [False] * (len(word_id[num]) - 1) for num in range(len(word_id))]
        mask = [idx for word in mask_nest for idx in word]

        return torch.tensor(word_ids_flat, dtype=torch.long), \
               torch.tensor(mask, dtype=torch.bool), \
               torch.tensor(lang_ids_flat, dtype=torch.long)

    def tensorify_all(self):
        new_data = []
        for elem in self.decoree:
            new_data.append(self.tensorify(elem))
        self.data = new_data

    def set_lang_to_idx(self, l_to_idx):
        self.lang_to_idx = l_to_idx
        self.decoree.lang_to_idx = l_to_idx

    def set_subword_to_idx(self, s_to_idx):
        self.subword_to_idx = s_to_idx
        self.decoree.char_to_idx = s_to_idx


# ----------------------------------------------------------------------------------------------------------------------
# Based on https://github.com/chrisvdweth/ml-toolkit/blob/master/pytorch/utils/data/text/dataset.py
class BatchSampler(Sampler):
    """
    This class creates batches containing equal length examples.
    """

    def __init__(self, batch_size, inputs):
        self.batch_size = batch_size
        self.input = inputs
        self.batch_list = self._generate_batch_map()
        self.num_batches = len(self.batch_list)

    def _generate_batch_map(self):
        batch_map = {}
        for idx, item in enumerate(self.input):
            length = len(item[0])
            if length not in batch_map:
                batch_map[length] = [idx]
            else:
                batch_map[length].append(idx)
        # Use batch_map to split indices into batches of equal size
        # e.g., for batch_size=3, batch_list = [[23,45,47], [49,50,62], [63,65,66], ...]
        batch_list = []
        for length, indices in batch_map.items():
            for group in [indices[i:(i + self.batch_size)] for i in range(0, len(indices), self.batch_size)]:
                batch_list.append(group)
        return batch_list

    def batch_count(self):
        return self.num_batches

    def __len__(self):
        return self.num_batches

    def __iter__(self):
        shuffle(self.batch_list)
        for i in self.batch_list:
            yield i

###functions for loading raw files and training the sentencepiece model


In [ ]:
import random
from collections import Counter
from typing import Optional
from torchtext.data import generate_sp_model
import re

# from datasets import Post
from pathlib import Path


def load_raw_post(filepath: Optional[Path]) -> list[Post]:
    """
    Reads the raw data from a file and converts it into a list of Post objects.
    This will later be shuffled and written into three separate files for training, development, and testing.
    :return: one list of Post objects
    """
    all_data = []
    words, langs = [], []

    if filepath is not None:
        with open(filepath) as file:
            for line in file:
                line = line.rstrip()
                if len(line) == 0:
                    if len(words) != 0:
                        all_data.append(Post(words, langs))
                        words, langs = [], []
                else:
                    word, lang, _ = line.split('\t')
                    if '+' in lang:
                        lang = 'mixed'
                    word = re.sub(r'(.)\1{2,}', r"\1\1", word.lower())
                    words.append(word)
                    langs.append(lang)
    return all_data

from sklearn.model_selection import train_test_split

def get_train_dev_test(data_filepath: str) -> tuple[list[Post], list[Post], list[Post]]:
    """
    Reads data from the provided directory path and splits the data into train, dev, and test
    :return: 3 lists of posts representing the train data, development data, and test data
    """
    # train = load_prepped_file(data_filepath + 'train.txt')
    # dev = load_prepped_file(data_filepath + 'dev.txt')
    # test = load_prepped_file(data_filepath + 'test.txt')

    train_data ,validation_data = train_test_split(dx,test_size=0.5)

    train = load_prepped_file(train_data)
    dev = load_prepped_file(validation_data)
    test = load_prepped_file(test_data)

    return train, dev, test


def write_prep_data(dirpath: str, data: tuple[list[Post], list[Post], list[Post]]) -> None:
    """
    Takes in raw train, dev and test data, and writes to a file with one post per line,
    with tokens and language tags split with a backslash
    :return: 3 lists of posts representing the train data, development data, and test data
    """

    fnames = [f'{dirpath}train.txt', f'{dirpath}dev.txt', f'{dirpath}test.txt']
    for fname, data_chunk in zip(fnames, data):
        with open(fname, 'w') as file:
            for post in data_chunk:
                tagged = ['/'.join((word, tag)) for word, tag in zip(post.words, post.langs)]
                file.write(" ".join(tagged) + '\n')


# def load_prepped_file(filepath: str) -> list[Post]:
#     """
#     Reads the prepared data from one file and converts it into a list of Post objects
#     :return: one list of Post objects
#     """
#     data = []
#     with open(filepath) as f:
#         for line in f:
#             line = line.rstrip().split(' ')
#             if len(line) > 0:
#                 words = [pair.rsplit('/', 1)[0] for pair in line]
#                 langs = [pair.rsplit('/', 1)[1] for pair in line]
#                 assert len(words) == len(langs)
#                 data.append(Post(words, langs))
#     return data

def load_prepped_file(df):
  data = []
  # print(df)
  for index, row in df.iterrows():
      # print(row)
      if len(row['words']) > 0:
          words = row['words']
          lid = row['lid']
          assert len(words) == len(lid)
          data.append(Post(words, lid))
  return data

def gen_sentpiece_model(vocab_size,
                        model_type: str = 'unigram',
                        train_filepath: str = '/content/drive/MyDrive/NLP_Project/train.txt',
                        output_filepath: str = '/content/drive/MyDrive/NLP_Project/sp_source_data.txt'):
    """
    Uses training data to generate sentencepiece source data. Creates sentencepiece model of provided
    vocab size and returns the sentencepiece model
    :return: sentencepiece model
    """
    # train_data = load_prepped_file(train_filepath)
    train_data = dx
    with open(output_filepath, 'w') as f:
        # for post in train_data:
        #     f.write(' '.join(post.words) + '\n')
        for row in dx['words']:
          # print(row)
          f.write(' '.join(row)+ '\n')

    generate_sp_model(output_filepath, vocab_size=vocab_size, model_prefix='./spm_user', model_type=model_type)


def print_stats(filepaths: list[Path]) -> None:
    langs = ['bn', 'en', 'univ', 'ne', 'hi', 'acro', 'mixed', 'undef']

    for f in filepaths:
        dataset: list[Post] = load_raw_post(f)
        lang_counts = Counter(lang for post in dataset for lang in post.langs)
        num_tokens = sum([len(post) for post in dataset])
        num_utts = str(len(dataset))
        lang_percs = "\t".join(["{:.2%}".format(lang_counts[lang] / num_tokens) for lang in langs])
        print(f'{f.name}\t\t{num_tokens}\t{num_utts}\t{lang_percs}')


def split_write_data(write_path, all_data: list[Post]) -> tuple[list[Post], list[Post], list[Post]]:
    random.shuffle(all_data)
    # 60:20:20 split
    twenty_perc = int(len(all_data) * 0.2)
    train_end = twenty_perc * 3
    test_end = train_end + twenty_perc

    train = all_data[: train_end]
    test = all_data[train_end: test_end]
    dev = all_data[test_end:]

    write_prep_data(write_path, (train, dev, test))
    return train, dev, test


def print_cmis(filepaths: list[Path]) -> None:
    for f in filepaths:
        dataset: list[Post] = load_raw_post(f)
        cmis = "\t".join(compute_cmi(dataset))
        print(f'{f.name}\t\t{cmis}')


def compute_cmi(dataset: list[Post]):
    lang_tags = {'bn', 'en', 'hi', 'mixed'}
    non_lang_tags = {'univ', 'acro', 'ne', 'undef'}

    all_cmis = []
    num_tokens = 0

    for post in dataset:
        num_tokens += len(post)
        lang_counts = Counter(lang for lang in post.langs if lang in lang_tags)
        non_lang_counts = Counter(tag for tag in post.langs if tag in non_lang_tags)

        if len(lang_counts) == 0:
            cmi = 0
        else:
            max_wi = lang_counts.most_common(1)[0][1]
            denom = len(post) - sum(non_lang_counts.values())
            cmi = 100 * (1 - (max_wi / denom))
        all_cmis.append(cmi)

    num_tokens = str(num_tokens)
    num_utts = str(len(dataset))
    all_cmi = "{:.4}".format(sum(all_cmis) / len(all_cmis))
    mixed_cmi = "{:.4}".format(sum(all_cmis) / (len(all_cmis) - all_cmis.count(0)))
    code_mix_perc = "{:.2%}".format((len(all_cmis) - all_cmis.count(0)) / len(all_cmis))

    return num_tokens, num_utts, all_cmi, mixed_cmi, code_mix_perc


def unique_tokens(data: list[Post]) -> dict[str, set[str]]:
    bn_wordset = {post.words[i] for post in data for i in range(len(post)) if post.langs[i] == 'bn'}
    en_wordset = {post.words[i] for post in data for i in range(len(post)) if post.langs[i] == 'en'}
    return {'bn': bn_wordset, 'en': en_wordset}


def perc_tok_overlap(data_comp: list[Post], data_test: list[Post]):
    comp_set = unique_tokens(data_comp)
    test_set = unique_tokens(data_test)

    for lang, words in test_set.items():
        overlap_words = words & comp_set[lang]
        perc_overlap = len(overlap_words) / len(words)
        print(lang, "{:.2%}".format(perc_overlap))


VOCAB_SIZE = 7000


# def main():
    # data_sources = sorted([path for path in Path('./data').iterdir()])
    # data = [post for f in data_sources for post in load_raw_post(f)]
    # # print_stats(data_sources)
    # # print_cmis(data_sources)
    # prep_path = './train_dev_test_data/'
    # split_write_data(prep_path, data)

    # train, dev, test = get_train_dev_test(prep_path)
    # print("Percentage of test tokens also in dev")
    # perc_tok_overlap(dev, test)
    # print("Percentage of test tokens also in train")
    # perc_tok_overlap(train, test)

gen_sentpiece_model(vocab_size=VOCAB_SIZE, model_type='unigram')


# if __name__ == "__main__":
#     main()

### full language ID model and training loop

In [ ]:
import time
import re
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

# from datasets import BatchSampler, PyTorchLIDDataSet
# from data_loading import VOCAB_SIZE


def correct_predictions(scores, masks, labels):
    pred = torch.argmax(scores, dim=1)
    masked_pred = torch.masked_select(pred, masks)
    masked_labels = torch.masked_select(labels, masks)
    return (masked_pred == masked_labels).sum()


class LIDModel(nn.Module):
    def __init__(self, subword_to_idx, lang_to_idx):
        # Subword_to_idx is a function that converts a subword to a number, and converts unknown tokens to 0
        # Lang_to_idx is a map that converts a language to a number
        self.subword_to_idx = subword_to_idx
        self.lang_to_idx = lang_to_idx
        self.idx_to_lang = dict([(value, key) for key, value in lang_to_idx.items()])
        self.vocab_size = VOCAB_SIZE
        self.lang_set_size = len(lang_to_idx)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        super(LIDModel, self).__init__()

    def pad_collate(self, batch):
        (words, masks, labels) = batch[0]
        words = words.to(self.device)
        masks = masks.to(self.device)
        labels = labels.to(self.device)
        return words, masks, labels

    def prepare_sentence(self, sentence: list[str]) -> tuple[torch.tensor, torch.tensor]:
        norm_sent = [re.sub(r'(.)\1{2,}', r"\1\1", word.lower()) for word in sentence]
        word_id = list(self.subword_to_idx([word for word in norm_sent]))
        mask_nest = [[True] + [False] * (len(word_id[num]) - 1) for num in range(len(word_id))]

        word_ids_flat = [w_id for word in word_id for w_id in word]
        mask_flat = [idx for word in mask_nest for idx in word]

        id_tensor = torch.tensor(word_ids_flat, dtype=torch.long, device=self.device).view(1, len(word_ids_flat))
        mask_tensor = torch.tensor(mask_flat, dtype=torch.bool, device=self.device)

        return id_tensor, mask_tensor

    def forward(self, sentence: list[str]):
        raise NotImplemented

    def predict(self, sentence: list[str]) -> list[tuple[str, str]]:
        self.eval()
        prep_sent, mask = self.prepare_sentence(sentence)

        feats = self(prep_sent).transpose(1, 2)  # shape (batch_size, seq_len, num_labels)
        feats_smax = F.softmax(feats, dim=-1).squeeze()  # softmaxing over each word

        preds = torch.argmax(feats_smax, dim=-1)
        masked_preds = torch.masked_select(preds, mask)
        lang_preds = [self.idx_to_lang[pred.item()] for pred in masked_preds]

        zipped_preds = [(word, lang) for word, lang in zip(sentence, lang_preds)]

        self.train()
        return zipped_preds
        # return Post(sentence, lang_preds)

    def rank(self, sentence: list[str]) -> dict[str: list[float]]:
        self.eval()
        prep_sent, mask = self.prepare_sentence(sentence)
        feats = self(prep_sent).transpose(1, 2)  # shape (batch_size, seq_len, num_labels)
        feats_smax = F.softmax(feats, dim=-1).squeeze()  # softmaxing over each word
        feats_smax = feats_smax.unsqueeze(0) if len(list(feats_smax.size())) < 2 else feats_smax

        lang_to_confs = {lang: [] for lang in self.lang_to_idx.keys()}
        for feat, f_mask in zip(feats_smax, mask):
            if f_mask:
                for lang, lang_idx in self.lang_to_idx.items():
                    lang_to_confs[lang].append(feat[lang_idx].item())
        self.train()
        return lang_to_confs

    def fit(self, train_dataset: PyTorchLIDDataSet, dev_dataset: PyTorchLIDDataSet,
            optimizer, epochs=3, batch_size=64, weight_dict=None):
        test_sampler = BatchSampler(batch_size, dev_dataset)
        dataloader_dev = DataLoader(dev_dataset,
                                    shuffle=False,
                                    drop_last=False,
                                    collate_fn=self.pad_collate,
                                    sampler=test_sampler)

        weights = None
        if weight_dict is not None:
            weights = torch.zeros(len(weight_dict)).to(self.device)
            for lang in weight_dict:
                indx = self.lang_to_idx[lang]
                weights[indx] = weight_dict[lang]

        best_dev = {'accuracy': 0, 'epoch': 0}

        # Index of the dummy label is -1
        loss_train = nn.CrossEntropyLoss(weight=weights, ignore_index=-1)
        loss_dev = nn.CrossEntropyLoss(ignore_index=-1)

        print(f"Running for {epochs} epochs")
        for epoch in range(epochs):
            self.train()
            avg_total_loss, num_correct_preds = 0, 0
            epoch_start_time = time.time()
            #
            print(batch_size)
            print(len(train_dataset))
            #
            sampler = BatchSampler(batch_size, train_dataset)
            dataloader_train = DataLoader(train_dataset,
                                          shuffle=False,
                                          drop_last=False,
                                          collate_fn=self.pad_collate,
                                          sampler=sampler)

            # Logit is the pre-softmax scores
            for idx, batch in enumerate(tqdm(dataloader_train, leave=False)):
                optimizer.zero_grad()
                tensor_sentences, masks, labels = batch
                logit = self(tensor_sentences)
                loss_nll = loss_train(logit, labels)
                num_correct_preds += correct_predictions(logit, masks, labels)
                loss = loss_nll
                avg_total_loss += loss.item()
                loss.backward()
                optimizer.step()
            avg_total_loss /= sampler.batch_count()

            train_num_tokens = sum(train_dataset.all_post_lens)
            accuracy = (num_correct_preds / train_num_tokens).item()

            print(f"\nAverage training error in epoch {epoch + 1}: {avg_total_loss:.5f} "
                  f"and training accuracy: {accuracy:.4f}")
            step_num = epoch
            print("Training Accuracy:", accuracy, step_num)
            print("Training Loss:", avg_total_loss, step_num)
            self.eval()
            # Test model
            avg_total_loss, num_correct_preds = 0, 0
            for _, batch in enumerate(tqdm(dataloader_dev, leave=False)):
                tensor_sentences, masks, labels = batch
                logit = self(tensor_sentences)
                loss_nll = loss_dev(logit, labels)
                num_correct_preds += correct_predictions(logit, masks, labels)
                avg_total_loss += loss_nll.item()
            avg_total_loss /= test_sampler.batch_count()

            dev_num_tokens = sum(dev_dataset.all_post_lens)
            accuracy = (num_correct_preds / dev_num_tokens).item()

            print(f"\nAverage total loss dev: {avg_total_loss:.5f}, accuracy: {accuracy:.4f}, ")
            print("Dev Accuracy:", accuracy, step_num)
            print("Dev Loss:", avg_total_loss, step_num)

            if accuracy > best_dev['accuracy']:
                best_dev['accuracy'] = accuracy
                best_dev['epoch'] = epoch
            if accuracy > 0.92:
                self.save_model("E" + str(epoch))
            print("Time spent in epoch {0}: {1:.2f} ".format(epoch + 1, time.time() - epoch_start_time))

        print(f"\nBest dev accuracy: {best_dev['accuracy']} found in EPOCH: {best_dev['epoch']}")

    def save_model(self, fileending=""):
        """Saves a pytorch model fully
        """
        required_model_information = {'model_state_dict': self.state_dict()}
        fname = "./trained_model_dict" + fileending + ".pth"
        torch.save(required_model_information, fname)

### LSTM model that initializes as layers, implements forward method

In [ ]:
import torch
import torch.nn as nn
# from lid_model import LIDModel


DROPOUT = 0.4


class LSTMLIDModel(LIDModel):
    def __init__(self, subword_to_idx, lang_to_idx, embedding_dim=300, hidden_dim=300, layers=1):
        super(LSTMLIDModel, self).__init__(subword_to_idx, lang_to_idx)
        self.hidden_dim = hidden_dim
        self.num_layers = layers
        self.embedding_dim = embedding_dim

        self.embedding = nn.Embedding(self.vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=layers, bidirectional=True, batch_first=True)
        self.linear = nn.Linear(hidden_dim * 2, self.lang_set_size)
        self.dropout = nn.Dropout(DROPOUT)
        self.to(self.device)

    def forward(self, sentence):
        embed = self.embedding(sentence)
        embed = self.dropout(embed)
        outputs, _ = self.lstm(embed)
        outputs = self.linear(outputs)
        return outputs.transpose(1, 2)

    def save_model(self, fileending=""):
        """Saves a dict containing statedict and other required model parameters and adds it as artifact
        Arguments:
        """
        required_model_information = {'model_state_dict': self.state_dict(),
                                      'embedding_dim': self.embedding_dim,
                                      'hidden_dim': self.hidden_dim,
                                      'layers': self.num_layers}
        fname = "/content/drive/MyDrive/NLP_Project/trained_LID_model" + fileending + ".pth"
        torch.save(required_model_information, fname)

### train_test_model.py       # functions for training and testing the model, used in run_training.py

In [ ]:
from torch import optim
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm

# from lid_model import LIDModel
# from datasets import LIDDataset, PyTorchLIDDataSet, Post
# from data_loading import get_train_dev_test
from typing import Optional


def test_model(data_set, model: LIDModel) -> tuple[list[Post], np.ndarray]:
    model.eval()
    lang_to_idx = model.lang_to_idx
    data_loader = DataLoader(data_set, batch_size=1)

    conf_mat = np.zeros((len(lang_to_idx), len(lang_to_idx)))
    all_tagged_sents = []

    for i, item in enumerate(tqdm(data_loader, leave=False)):
        tokens, langs = item
        tokens = [w[0] for w in tokens]
        gold_labels = [lang[0] for lang in langs]
        pred_labels = [label for _, label in model.predict(tokens)]
        all_tagged_sents.append(Post(tokens, pred_labels))

        for pred, gold in zip(pred_labels, gold_labels):
            conf_mat[lang_to_idx[pred]][lang_to_idx[gold]] += 1

    return all_tagged_sents, conf_mat


def save_metrics(conf_mat, lang_labels):
    """Saves metrics as a .txt file
    Arguments:
        conf_mat  -- confusion matrix across dataset
        lang_labels -- list of language labels corresponding with confusion matrix heading
    """
    accuracy = conf_mat.trace() / np.sum(conf_mat)
    precision = np.diag(conf_mat) / np.sum(conf_mat, axis=1)
    recall = np.diag(conf_mat) / np.sum(conf_mat, axis=0)
    f1 = (2 * precision * recall) / (precision + recall)

    fname = "/content/drive/MyDrive/NLP_Project/test_metrics.txt"
    with open(fname, 'w') as f:
        f.write("\nAccuracy:\t" + "\t".join(["{:.4%}".format(accuracy)]))
        f.write("\nLabels:\t" + "\t".join(lang_labels))
        f.write("\nPrecision:\t" + "\t".join(["{:.4%}".format(prec) for prec in precision.tolist()]))
        f.write("\nRecall:\t" + "\t".join(["{:.4%}".format(rec) for rec in recall.tolist()]))
        f.write("\nF1:\t" + "\t".join(["{:.4%}".format(f) for f in f1.tolist()]))
        f.close()


def save_preds(predictions: list[Post]):
    """Saves predictions as a .txt file
    Arguments:
        preds  -- list of tagged sentences to save
    """
    fname = "/content/drive/MyDrive/NLP_Project/test_predictions.txt"
    with open(fname, 'w') as f:
        for post in predictions:
            tagged = ['/'.join((word, tag)) for word, tag in zip(post.words, post.langs)]
            f.write(" ".join(tagged) + "\n")
        f.close()


def train_model(data_set: PyTorchLIDDataSet, test_dataset: PyTorchLIDDataSet, lidmodel: 'LIDModel',
                training_params, weight_dict: Optional[dict] = None):
    optimizer, weight_decay, lr, batch_size, epochs = training_params
    if optimizer.strip().lower() == "sgd":
        opti = optim.SGD(lidmodel.parameters(), lr=lr, weight_decay=weight_decay)
    else:
        opti = optim.AdamW(params=lidmodel.parameters())
    lidmodel.fit(data_set, test_dataset, opti, epochs=epochs, weight_dict=weight_dict, batch_size=batch_size)


def run_training(model, training_params, to_train=True, eval_on_test=False):

    train, dev, test = get_train_dev_test('/content/drive/MyDrive/NLP_Project/')

    train_dataset = LIDDataset(train)
    dev_dataset = LIDDataset(dev)
    test_dataset = LIDDataset(test)

    train_data_converted = PyTorchLIDDataSet(train_dataset)
    dev_converted = PyTorchLIDDataSet(dev_dataset)

    weight_dict = train_dataset.weight_dict
    print(len(train_data_converted))
    print(len(dev_converted))
    if to_train:
        print("Training model")
        train_model(train_data_converted, dev_converted, model,
                    training_params=training_params, weight_dict=weight_dict)

    print("Testing model")
    dataset_for_eval = test_dataset if eval_on_test else dev_dataset
    predictions, confusion_mat = test_model(data_set=dataset_for_eval, model=model)

    print("Saving model")
    model.save_model()

    print("Saving predictions")
    save_preds(predictions)
    save_metrics(confusion_mat, list(dataset_for_eval.lang_to_idx.keys()))

### adjust hyperparameters and train the full model

In [ ]:
# Code is based on https://github.com/AU-DIS/LSTM_langid
from typing import Optional, Callable

import torch
from torchtext.data import sentencepiece_numericalizer, load_sp_model
# from train_test_model import run_training
# from lstm_model import LSTMLIDModel


def load_LSTM_model(pretrained_model_path: Optional[str],
                    subword_to_idx: Callable,
                    lang_to_idx: dict,
                    hidden_dim,
                    embedding_dim,
                    num_lstm_layers):
    if pretrained_model_path is not None:
        model_dict = torch.load(pretrained_model_path)
        lstm_model = LSTMLIDModel(subword_to_idx=subword_to_idx,
                                  lang_to_idx=lang_to_idx,
                                  hidden_dim=model_dict['hidden_dim'],
                                  embedding_dim=model_dict['embedding_dim'],
                                  layers=model_dict['layers'])
        lstm_model.load_state_dict(model_dict['model_state_dict'])

    else:
        lstm_model = LSTMLIDModel(subword_to_idx=subword_to_idx,
                                  lang_to_idx=lang_to_idx,
                                  hidden_dim=hidden_dim,
                                  embedding_dim=embedding_dim,
                                  layers=num_lstm_layers)
    return lstm_model


def main(pretrained_model,
         epochs,
         weight_decay,
         batch_size,
         lr,
         optimizer,
         eval_on_test=False):
    training_params = optimizer, weight_decay, lr, batch_size, epochs
    numericalizer = sentencepiece_numericalizer(load_sp_model('/content/spm_user.model'))
    # lang_to_idx = {'bn': 0, 'en': 1, 'univ': 2, 'ne': 3, 'hi': 4, 'acro': 5, 'mixed': 6, 'undef': 7}
    lang_to_idx = {'lang1':0 , 'lang2': 1}

    lstm_model = load_LSTM_model(pretrained_model_path=pretrained_model,
                                 subword_to_idx=numericalizer,
                                 lang_to_idx=lang_to_idx,
                                 hidden_dim=HIDDEN_DIM,
                                 embedding_dim=EMBEDDING_DIM,
                                 num_lstm_layers=NUM_LSTM_LAYERS)
    to_train = pretrained_model is None

    run_training(lstm_model, training_params, to_train, eval_on_test)


# PRETRAINED_MODEL = '/content/drive/MyDrive/NLP_Project/trained_LID_model.pth'
PRETRAINED_MODEL = None

EPOCHS = 40
SEED = 42
HIDDEN_DIM = 400
EMBEDDING_DIM = 400
NUM_LSTM_LAYERS = 1
OPTIMIZER = 'adam'
LR = 0.1
WEIGHT_DECAY = 0.00001
BATCH_SIZE = 64
EVAL_ON_TEST = True


# if __name__ == "__main__":
print(f'PRETRAINED_MODEL = {PRETRAINED_MODEL}\n'
      f'EPOCHS = {EPOCHS}\n'
      f'HIDDEN_DIM = {HIDDEN_DIM}\n'
      f'EMBEDDING_DIM = {EMBEDDING_DIM}\n'
      f'NUM_LSTM_LAYERS = {NUM_LSTM_LAYERS}\n'
      f'OPTIMIZER = {OPTIMIZER}\n'
      # f'LR = {LR}\n'
      # f'WEIGHT_DECAY = {WEIGHT_DECAY}\n'
      f'BATCH_SIZE = {BATCH_SIZE}')
main(pretrained_model=PRETRAINED_MODEL,
      epochs=EPOCHS,
      weight_decay=WEIGHT_DECAY,
      batch_size=BATCH_SIZE,
      lr=LR,
      optimizer=OPTIMIZER,
      eval_on_test=EVAL_ON_TEST)

PRETRAINED_MODEL = None
EPOCHS = 40
HIDDEN_DIM = 400
EMBEDDING_DIM = 400
NUM_LSTM_LAYERS = 1
OPTIMIZER = adam
BATCH_SIZE = 64
2407
2410
Training model
Running for 40 epochs
64
2407



Average training error in epoch 1: 0.26814 and training accuracy: 0.8869
Training Accuracy: 0.8868783712387085 0
Training Loss: 0.2681361338273235 0



Average total loss dev: 0.19690, accuracy: 0.9210, 
Dev Accuracy: 0.9209907650947571 0
Dev Loss: 0.19689608715988438 0
Time spent in epoch 1: 41.02 
64
2407



Average training error in epoch 2: 0.14857 and training accuracy: 0.9274
Training Accuracy: 0.9273706674575806 1
Training Loss: 0.14857154116068497 1



Average total loss dev: 0.18114, accuracy: 0.9338, 
Dev Accuracy: 0.9338282346725464 1
Dev Loss: 0.18113667231277916 1
Time spent in epoch 2: 39.07 
64
2407


...


Average training error in epoch 39: 0.00504 and training accuracy: 0.9977
Training Accuracy: 0.9977350234985352 38
Training Loss: 0.005041960870259204 38



Average total loss dev: 0.24214, accuracy: 0.9597, 
Dev Accuracy: 0.9596991539001465 38
Dev Loss: 0.24213970284760197 38
Time spent in epoch 39: 41.33 
64
2407



Average training error in epoch 40: 0.00304 and training accuracy: 0.9983
Training Accuracy: 0.9982635378837585 39
Training Loss: 0.0030395434670973842 39



Average total loss dev: 0.24040, accuracy: 0.9614, 
Dev Accuracy: 0.9613651037216187 39
Dev Loss: 0.24039900596630795 39
Time spent in epoch 40: 37.77 

Best dev accuracy: 0.9613651037216187 found in EPOCH: 39
Testing model


Saving model
Saving predictions


### Language identifier Predictons

In [ ]:
from torchtext.data import sentencepiece_numericalizer, load_sp_model, sentencepiece_tokenizer

# from lstm_model import LSTMLIDModel # lstm_model.py
from nltk import TreebankWordTokenizer
import torch
from pathlib import Path


class LanguageIdentifier:
    """
    Word-level language identifier
    """
    def __init__(self, directory_path: Path = "/content/drive/MyDrive/NLP_Project/trained_LID_model.pth"):
        model_information_dict = torch.load(directory_path)
        subword_to_idx = sentencepiece_numericalizer(load_sp_model('/content/spm_user.model'))
        # lang_to_idx = {'bn': 0, 'en': 1, 'univ': 2, 'ne': 3, 'hi': 4, 'acro': 5, 'mixed': 6, 'undef': 7}
        lang_to_idx = {'lang1':0 , 'lang2': 1}
        self.model = LSTMLIDModel(subword_to_idx, lang_to_idx,
                                  model_information_dict['embedding_dim'],
                                  model_information_dict['hidden_dim'],
                                  model_information_dict['layers'])
        self.model.load_state_dict(model_information_dict['model_state_dict'], strict=False)
        self.tokenizer = TreebankWordTokenizer()

    def tokenize(self, input_sentence: str) -> list[str]:
        return self.tokenizer.tokenize(input_sentence)

    def predict(self, input_sentence: str) -> list[tuple[str, str]]:
        tokens = self.tokenize(input_sentence)
        return self.model.predict(tokens)

    def rank(self, input_sentence: str) -> dict[str, list]:
        tokens = self.tokenize(input_sentence)
        return self.model.rank(tokens)


# To instantiate an instance of the language identifier
LID = LanguageIdentifier()
ex_sent = "Took Strewn out for a stroll today 🖤 @happybaby"
print(LID.predict(ex_sent))

[('Took', 'lang1'), ('Strewn', 'lang1'), ('out', 'lang1'), ('for', 'lang1'), ('a', 'lang1'), ('stroll', 'lang1'), ('today', 'lang1'), ('🖤', 'lang1'), ('@', 'lang1'), ('happybaby', 'lang1')]


In [ ]:
from nltk.corpus import words

In [ ]:
def foo(text):
  out = LID.predict(text)
  # print(out)
  out = list(zip(*out))
  # print(out)
  result = []
  for x in range(len(out[0])):
      # print(out[0][x])
    if out[1][x] != 'lang1':
      if out[0][x] not in words.words():
          return ''
    else:
      result.append(out[0][x])
  # print('\n')
  # print(result)
  return ' '.join(result)
ex_sent = "i remember moments after giving birth to this kid i thought to myself i have no idea what i m doing but i figured since i had kept my cats alive for 15 years that i d be ok grinning squinting face turns out taking care of newborns and babies in general isn t too difficult if your used to 0 sleep person raising hand person raising hand person raising hand it s when they start testing you with their witty comebacks that you start wishing you would have read one of those parenting books the chaos of toddlerhood is more real now than ever but look at that happy little guy there s still a little baby face left there but not for much longer i m a learn as i go type mama and we are figuring out this toddler stuff together making memories and living our best life that s what it s all about person tipping hand medium light skin tone bullseye bullseye now on our way to bullseye bullseye bullseye	"
foo(ex_sent)

''

In [ ]:
likable_data = pd.read_csv('/content/drive/MyDrive/ML Caption Craft/likable_dataV2.csv', lineterminator='\n')

In [ ]:
likable_data.head(10)

,Unnamed: 0,img_p,caption,preprocessed_captions
0,0,1861903620333890097.jpg,"“You can be whatever size you are, and you can be beautiful both inside and out” - Serena\n.\nWhat’s your S? “Mines is First Day Of SCHOOL!!” 💪🏾📚🎒\n@serenawilliams @serena #BeSeenBeHeard #backtoschool I #love #serenawilliams",you can be whatever size you are and you can be beautiful both inside and out serena what is your s mines is first day of school i
1,1,1453987990896372228.jpg,@sportsillustrated night deux ✨,night deux
2,2,1532896425460868252.jpg,BASIC • @adidas,basic
3,3,2020784155847137387.jpg,I may feel like crapola today but at least I can still show a little leg and feel good about myself 💁🏼 💎 Find this chambray dress at @target I am wearing a size XXL for some extra wiggle room. Fringe booties are target last season but I’ve linked similar in my amazon shop! Link in bio!,i may feel like crapola today but at least i can still show a little leg and feel good about myself find this chambray dress at i am wearing a size xxl for some extra wiggle room fringe booties are target last season but i have linked similar in my amazon shop link in bio
4,4,2022223424923098097.jpg,🍃You know what I am? 🍃I am smart 🍃I am talented 🍃I take advantage of the opportunities that come my way ☝️AND I work really really hard. 👊\nDon’t call me lucky. Call me a badass 💋\n.\n.\n💎Nude cardi on sale @justfabonline and dress is @nphilanthropy available in my amazon shop link in bio.,you know what i am i am smart i am talented i take advantage of the opportunities that come my way and i work really really hard do not call me lucky call me a badass nude cardi on sale and dress is available in my amazon shop link in bio
5,5,2022602597378110298.jpg,I remember moments after giving birth to this kid I thought to myself — I have no idea what I’m doing!?!? .. but I figured since I had kept my cats alive for 15 years that I’d be ok 😆Turns out taking care of newborns and babies in general isn’t too difficult if your used to 0 sleep 🙋🙋🙋 It’s when they start testing you with their witty comebacks that you start wishing you would have read one of those parenting books. The chaos of toddlerhood is more real now than ever. But look at that happy little guy— there’s still a little baby face left there but not for much longer. I’m a learn as I go type Mama and we are figuring out this toddler stuff together. Making memories and living our best life. That’s what it’s all about. 💁🏼\n🎯🎯Now on our way to @target !!! 🎯🎯🎯,i remember moments after giving birth to this kid i thought to myself i have no idea what i am doing but i figured since i had kept my cats alive for 15 years that i would be ok turns out taking care of newborns and babies in general is not too difficult if your used to 0 sleep it is when they start testing you with their witty comebacks that you start wishing you would have read one of those parenting books the chaos of toddlerhood is more real now than ever but look at that happy little guy there is still a little baby face left there but not for much longer i am a learn as i go type mama and we are figuring out this toddler stuff together making memories and living our best life that is what it is all about now on our way to
6,6,2024142324329636087.jpg,"🌷🐚🌱Mary, Mary, quite contrary\nHow does your garden grow?\nWith silver bells and cockleshells\nAnd pretty maids all in a row. 🌱🐚🌷\n.\n.\n💎 Dress by @targetstyle #anewday and it was only $15 — I purchased 3 days ago so it’s still on racks! Perfect for Easter! .\n.\n#springtime #springstyle #springfashion2019 #springfashion #nurseryrhymes #marymaryquitecontrary #angieknowsthetruth #gardenfashion #springvibes #sunnydays #plussize #plussizefashion #morningglow #blooming #springmorning #targetstyle #targetfashion #targetdoesitagain #targetforthewin #cincinnatifashion",mary mary quite contrary how does your garden grow with silver bells and cockleshells and pretty maids all in a row dress by and it was only 15 i purchased 3 days a

In [ ]:
likable_data['non_english_captions'] = likable_data['preprocessed_captions'].progress_apply(lambda x:foo(x))

100%|██████████| 23933/23933 [15:37<00:00, 25.54it/s]


In [ ]:
likable_data[likable_data['non_english_captions']== ''].shape #8385 #5400

(5796, 5)

In [ ]:
likable_data[likable_data['non_english_captions']== '']

,Unnamed: 0,img_p,caption,preprocessed_captions,non_english_captions
11,11,1968601135167541360.jpg,"Keep reading for a fun giveaway with @onesmallblonde! Head to her profile to enter to win some awesome home gym essentials 🏋️‍♀️🏋️‍♂️\n.\n#Repost @onesmallblonde\n・・・\nGIVEAWAY TIME! 💪🏻 We are back with a new ""Physically FIIT"" post sharing a full-body at home workout with the best gym essentials from @academy! @spencerconley_ did not take it easy on me with this one... 🙈 But he knows how to push me just enough to get in the best shape! All of these workouts are REAL moves I do in the gym with my trainer 👌🏻 Now when I'm too busy to get to the gym I can still get a workout in at home! No excuses to skip a workout anymore... ⠀⠀⠀⠀⠀⠀⠀⠀⠀\nEnter to WIN this complete gym essentials set + my workout tank and leggings from @adidas! 🖤💪🏻\n⠀⠀⠀⠀⠀⠀⠀⠀⠀\nHOW TO ENTER:\n• Follow me and @academy\n• Comment below using #sweeps\n• Tag 1 friend\n⠀⠀⠀⠀⠀⠀⠀⠀⠀\nEnds February 2nd at 11:59pm CST! Good luck!! Full giveaway details here >> https://bit.ly/2WviVoH #academysports #fitnessblog #athomeworkouts #gymequipment #physicallyfit #adidaswomen #dallasfitness",keep reading for a fun giveaway with head to her profile to enter to win some awesome home gym essentials giveaway time we are back with a new physically fiit post sharing a full body at home workout with the best gym essentials from did not take it easy on me with this one but he knows how to push me just enough to get in the best shape all of these workouts are real moves i do in the gym with my trainer now when i am too busy to get to the gym i can still get a workout in at home no excuses to skip a workout anymore enter to win this complete gym essentials set my workout tank and leggings from how to enter follow me and comment below using tag 1 friend ends february 2nd at 11 59pm cst good luck full giveaway details here,
14,14,1988516875491003760.jpg,":: impara a fare l'hummus e conquisterai qualunque carnivoro ::⁣\nmi ricordo ancora di quando ho iniziato a eliminare la carne. avevo la nostalgia del suo gusto. vivevo in inghilterra e tutti i giorni per tornare a casa passavo davanti a una catena di pollo arrosto. e l'odore mi faceva venire voglia e l'acquolina in bocca ad ogni passaggio. è stato molto difficile all'inizio. e ho capito che la cosa che mancava al mio palato erano i sapori forti. quelli corposi e sostanziosi. mentre la cucina vegetariana era molto più delicata e il mio palato non era così delicato da riuscire ad apprezzarli. così utilizzavo le spezie. cumino nell'hummus e nell'impasto del pane. curry nel dhal. garam masala nella zuppa di ceci in stile indiano. e così via. il mio palato si placava. le mie voglie anche. e riuscivo ad avere la tranquillità di non essere tentata da un piatto di grigliata (sì, per me ci è voluta tanta forza di volontà!). poi dopo qualche mese, il mio palato si è abituato a questi sapori meno forti, ma più profondi. ho iniziato ad apprezzare la delicatezza e la diversità di sapore. ho diminuito le spezie, che però continuo a usare in dosi inferiori per insaporire i piatti. ⁣\nuna delle ricette che accontenta tutti i miei amici non veg è il mio hummus. richiesto a gran voce a qualsiasi festa e compleanno. l'ho preparato anche in occasione del pranzo social organizzato al ristorante Il Birichin di @nicolabatavia. e credo che tutti abbiano gradito.⁣\nè stata una versione semplificata dell'hummus, perchè mi mancavano alcuni ingredienti di battaglia (tipo il mio amato tahin!). l'ho ottenuto semplicemente mescolando la crema di ceci che lo chef aveva già preparato, del succo di limone, sale e pepe, un pizzico di spezie (tra cui anche il cumino) e dell'olio di oliva. ⁣\n::⁣\n#plantbased #plantbasedlife #pranzo #lestradeditorino #torino #torinoèlamiacittà #eatplants #pranzosano #pranzosalutare #vegano #eataly #italianfood #ciboitaliano",impara a fare l hummus e conquisterai qualunque carnivoro mi ricordo ancora di quando ho iniziato a eliminare la carne avevo la

In [ ]:
likable_data = likable_data[likable_data['non_english_captions'] != '' ]
likable_data.shape

(18137, 5)

In [ ]:
likable_data =likable_data[['img_p'	,'caption',	'preprocessed_captions']]
likable_data.head()

,img_p,caption,preprocessed_captions
0,1861903620333890097.jpg,"“You can be whatever size you are, and you can be beautiful both inside and out” - Serena\n.\nWhat’s your S? “Mines is First Day Of SCHOOL!!” 💪🏾📚🎒\n@serenawilliams @serena #BeSeenBeHeard #backtoschool I #love #serenawilliams",you can be whatever size you are and you can be beautiful both inside and out serena what is your s mines is first day of school i
1,1453987990896372228.jpg,@sportsillustrated night deux ✨,night deux
2,1532896425460868252.jpg,BASIC • @adidas,basic
3,2020784155847137387.jpg,I may feel like crapola today but at least I can still show a little leg and feel good about myself 💁🏼 💎 Find this chambray dress at @target I am wearing a size XXL for some extra wiggle room. Fringe booties are target last season but I’ve linked similar in my amazon shop! Link in bio!,i may feel like crapola today but at least i can still show a little leg and feel good about myself find this chambray dress at i am wearing a size xxl for some extra wiggle room fringe booties are target last season but i have linked similar in my amazon shop link in bio
4,2022223424923098097.jpg,🍃You know what I am? 🍃I am smart 🍃I am talented 🍃I take advantage of the opportunities that come my way ☝️AND I work really really hard. 👊\nDon’t call me lucky. Call me a badass 💋\n.\n.\n💎Nude cardi on sale @justfabonline and dress is @nphilanthropy available in my amazon shop link in bio.,you know what i am i am smart i am talented i take advantage of the opportunities that come my way and i work really really hard do not call me lucky call me a badass nude cardi on sale and dress is available in my amazon shop link in bio


In [ ]:
likable_data.to_csv('/content/drive/MyDrive/ML Caption Craft/clean_data.csv', lineterminator='\n')

In [ ]:
likable_data = pd.read_csv('/content/drive/MyDrive/ML Caption Craft/clean_data.csv', lineterminator='\n')

In [ ]:
likable_data.head()

,Unnamed: 0,img_p,caption,preprocessed_captions
0,0,1861903620333890097.jpg,"“You can be whatever size you are, and you can be beautiful both inside and out” - Serena\n.\nWhat’s your S? “Mines is First Day Of SCHOOL!!” 💪🏾📚🎒\n@serenawilliams @serena #BeSeenBeHeard #backtoschool I #love #serenawilliams",you can be whatever size you are and you can be beautiful both inside and out serena what is your s mines is first day of school i
1,1,1453987990896372228.jpg,@sportsillustrated night deux ✨,night deux
2,2,1532896425460868252.jpg,BASIC • @adidas,basic
3,3,2020784155847137387.jpg,I may feel like crapola today but at least I can still show a little leg and feel good about myself 💁🏼 💎 Find this chambray dress at @target I am wearing a size XXL for some extra wiggle room. Fringe booties are target last season but I’ve linked similar in my amazon shop! Link in bio!,i may feel like crapola today but at least i can still show a little leg and feel good about myself find this chambray dress at i am wearing a size xxl for some extra wiggle room fringe booties are target last season but i have linked similar in my amazon shop link in bio
4,4,2022223424923098097.jpg,🍃You know what I am? 🍃I am smart 🍃I am talented 🍃I take advantage of the opportunities that come my way ☝️AND I work really really hard. 👊\nDon’t call me lucky. Call me a badass 💋\n.\n.\n💎Nude cardi on sale @justfabonline and dress is @nphilanthropy available in my amazon shop link in bio.,you know what i am i am smart i am talented i take advantage of the opportunities that come my way and i work really really hard do not call me lucky call me a badass nude cardi on sale and dress is available in my amazon shop link in bio


In [ ]:
likable_data.shape

(18137, 4)

In [ ]:
likable_data = likable_data[likable_data.preprocessed_captions.map(lambda x: x.isascii())]

In [ ]:
likable_data.shape

(17237, 4)

In [ ]:
!pip install contractions
# !pip install num2words
# !pip install -U scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 4.3 MB/s eta 0:00:00


In [ ]:
import contractions
def remove_html_tags(data):
    soup = BeautifulSoup(data, 'html.parser')
    return soup.get_text()

def clean_text(text):

    text = str(text).lower()
    text = remove_html_tags(text)
    text =  contractions.fix(text)
    text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ',text)
    text = re.sub('[^a-zA-Z]', ' ' , text)
    return text
likable_data['preprocessed_captions'] = likable_data['preprocessed_captions'].apply(lambda x:clean_text(x))

In [ ]:
likable_data.shape

(17237, 4)

In [ ]:
likable_data.to_csv('/content/drive/MyDrive/ML Caption Craft/clean_data.csv', lineterminator='\n')

In [ ]:
likable_data.iloc[9346]

Unnamed: 0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           12940
img_p                                                                                                                                        